# why RNN

there  is no such n/w which works on sequential data
 activation used is TANH OR RELU

 it works in  language translation,sentiment analysis etc

 1] many to many = no.of input - no.of output = usecase = speech recognization
 2] many to one = many input - one output = usecase = sentiment analysis
 3] input diff - output diff(sequence - sequence) = usecase = language translat
 4] one input - many output = usecase = music genere 

In [0]:
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/database.sqlite.zip', 'r') as zip_ref:
  zip_ref.extractall('')

In [0]:
import cv2
import numpy as np

In [0]:
import sqlite3
import pandas as pd
#create your connection
sqlite = sqlite3.connect('/content/database.sqlite')
df = pd.read_sql_query("SELECT  *  FROM SENTIMENT ", sqlite )
#c = conn.cursor()

In [0]:
df

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,name,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,,I_Am_Kenzi,,5,,,RT @NancyLeeGrahn: How did everyone feel about...,,2015-08-07 09:54:46 -0700,629697200650592256,,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,,PeacefulQuest,,26,,,RT @ScottWalker: Didn't catch the full #GOPdeb...,,2015-08-07 09:54:46 -0700,629697199560069120,,
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,,PussssyCroook,,27,,,RT @TJMShow: No mention of Tamir Rice and the ...,,2015-08-07 09:54:46 -0700,629697199312482304,,
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,,MattFromTexas31,,138,,,RT @RobGeorge: That Carly Fiorina is trending ...,,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,,sharonDay5,,156,,,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,,2015-08-07 09:54:45 -0700,629697196967903232,,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13866,13867,No candidate mentioned,1.0000,yes,1.0000,Negative,0.7991,Abortion,0.6014,No candidate mentioned,SantoliDonato,yes,7,Negative,Abortion\nWomen's Issues (not abortion though),RT @cappy_yarbrough: Love to see men who will ...,,2015-08-07 09:29:43 -0700,629690895479250944,Como,
13867,13868,Mike Huckabee,0.9611,yes,1.0000,Positive,0.7302,None of the above,0.9229,Mike Huckabee,mhfa16hq,yes,1,,,RT @georgehenryw: Who thought Huckabee exceede...,,2015-08-07 09:25:02 -0700,629689719056568320,USA,
13868,13869,Ted Cruz,1.0000,yes,1.0000,Positive,0.8051,None of the above,0.9647,Ted Cruz,DrottM,yes,67,Positive\nNeutral,,"RT @Lrihendry: #TedCruz As President, I will a...",,2015-08-07 07:19:18 -0700,629658075784282112,,
13869,13870,Donald Trump,1.0000,yes,1.0000,Negative,1.0000,Women's Issues (not abortion though),0.9202,Donald Trump,danijeantheq,yes,149,,Women's Issues (not abortion though),RT @JRehling: #GOPDebate Donald Trump says tha...,,2015-08-07 09:54:04 -0700,629697023663546368,,


In [0]:
# this python 3 environment comes with many helpful analytics libraries installed
# it is defined by the kaggle/python docker image
#for example, here is several helpful packages to load in


import numpy as np #linear algebra
import pandas as pd   #data processing,CSV file I/0 (ex : pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer   #convert text to vector form
from tensorflow.keras.preprocessing.text import Tokenizer   #convert whole sentence into a unique word
from tensorflow.keras.preprocessing.sequence import pad_sequences   #it will make the fixed dimensions for each sentences
from tensorflow.keras.models import Sequential                       # ^ it will pad for small dimension sentence
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D   #spatialdrop1D for LSTM
from sklearn.model_selection import train_test_split           #^ embedding layer will learn which category will assign to vector form
from keras.utils.np_utils import to_categorical    #ex car,bike,bus if will learn car it will convert to vector        
from sklearn.preprocessing import LabelEncoder
import re         #regluration expression ex:instead of text we use symbol here

In [0]:
#data = pd.read_csv(.....)
#keeping only necessory columns

data = df[['text', 'sentiment']]

In [0]:
set(data['sentiment'])

{'Negative', 'Neutral', 'Positive'}

In [0]:
data = data[data.sentiment != "Neutral"] #if you want to check for specific one if you want to check for all
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-Z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)
print(data[ data['sentiment'] == 'Neutral'].size)

for idx,row in data.iterrows():
  #print(idx,row)
  row[0] = row[0].replace('rt', ' ')

max_features = 5000
tokenizer = Tokenizer(num_words = max_features, split = ' ')   #how many words we want to take to your vocabulary
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)    #converting to vector
X = pad_sequences(X)


4472
16986
0


In [0]:
X.shape

(10729, 29)

defining our LSTM model

In [0]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout = 0.2, recurrent_dropout=0.5))
model.add(Dense(2, activation = 'softmax'))      #this dense depend upon output(positive,negative,nuetral)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 29, 128)           640000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 29, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 394       
Total params: 895,194
Trainable params: 895,194
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
Y = pd.get_dummies(data['sentiment']).values
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape, Y_train.shape)
print(X_test.shape,Y_test.shape)

(7188, 29) (7188, 2)
(3541, 29) (3541, 2)


In [0]:
batch_size = 32
model.fit(X_train, Y_train, epochs= 5, batch_size = batch_size, verbose = 1, validation_data=(X_test,Y_test))

Train on 7188 samples, validate on 3541 samples
Epoch 1/5
7188/7188 [==============================] - 16s 2ms/sample - loss: 0.4524 - acc: 0.8109 - val_loss: 0.3632 - val_acc: 0.8430
Epoch 2/5
7188/7188 [==============================] - 15s 2ms/sample - loss: 0.3164 - acc: 0.8678 - val_loss: 0.3440 - val_acc: 0.8608
Epoch 3/5
7188/7188 [==============================] - 15s 2ms/sample - loss: 0.2573 - acc: 0.8975 - val_loss: 0.3484 - val_acc: 0.8520
Epoch 4/5
7188/7188 [==============================] - 15s 2ms/sample - loss: 0.2212 - acc: 0.9114 - val_loss: 0.3979 - val_acc: 0.8594
Epoch 5/5
7188/7188 [==============================] - 15s 2ms/sample - loss: 0.1793 - acc: 0.9307 - val_loss: 0.3943 - val_acc: 0.8577


In [0]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]

X_test = X_test[: -validation_size]
Y_test = Y_test[: -validation_size]

score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)

print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

2041/2041 - 1s - loss: 0.3953 - acc: 0.8535
score: 0.40
acc: 0.85


In [0]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0,0,0,0
for x in range(len(X_validate)):

  result = model.predict(X_validate[x].reshape(1,X_test.shape[1]), batch_size=1, verbose = 2)[0]

  if np.argmax(result) == np.argmax(Y_validate[x]):

    if np.argmax(Y_validate[x]) == 0:
      neg_correct += 1
    else:
      pos_correct += 1

  if np.argmax(Y_validate[x]) == 0:
    neg_cnt += 1
  else:
    pos_cnt += 1

1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1

In [0]:
print("pos_acc", pos_correct/pos_cnt*100,"%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 59.22330097087378 %
neg_acc 93.36691855583543 %


In [0]:
twt = ["Meetings: Because none of us is as dumb as all of us."]
twt = ['I like the movie it is amazing good']          #in the tweets what matter we give it will calculate and gives output

#vectorizing the tweet by the pre-fitted tokenzier instance
twt = tokenizer.texts_to_sequences(twt) 

#padding the tweet to have exactly the same shape as embedding_2 input

twt = pad_sequences(twt, maxlen = 29, dtype = 'int32', value = 0)
print(twt)

sentiment = model.predict(twt, batch_size = 1, verbose = 2)[0]

if(np.argmax(sentiment) == 0):
  print("negative")

elif (np.argmax(sentiment) == 1):
  print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   9  38   1 951  13   5 924 137]]
1/1 - 0s
positive


In [0]:
twt = ["yor are not looking good"]
#in the tweets what matter we give it will calculate and gives output

#vectorizing the tweet by the pre-fitted tokenzier instance
twt = tokenizer.texts_to_sequences(twt) 

#padding the tweet to have exactly the same shape as embedding_2 input

twt = pad_sequences(twt, maxlen = 29, dtype = 'int32', value = 0)
print(twt)

sentiment = model.predict(twt, batch_size = 1, verbose = 2)[0]

if(np.argmax(sentiment) == 0):
  print("negative")

elif (np.argmax(sentiment) == 1):
  print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0  36  19 287 137]]
1/1 - 0s
negative


In [0]:
twt = ["you are not looking bad"]
#in the tweets what matter we give it will calculate and gives output

#vectorizing the tweet by the pre-fitted tokenzier instance
twt = tokenizer.texts_to_sequences(twt) 

#padding the tweet to have exactly the same shape as embedding_2 input

twt = pad_sequences(twt, maxlen = 29, dtype = 'int32', value = 0)
print(twt)

sentiment = model.predict(twt, batch_size = 1, verbose = 2)[0]

if(np.argmax(sentiment) == 0):
  print("negative")

elif (np.argmax(sentiment) == 1):
  print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0  11  36  19 287 342]]
1/1 - 0s
negative
